# LCA contribution analysis

This notebook does LCA calculations for all impact categories (but not including uncertainty), writing the results to `results/all_impact_category_contributions.csv`.  Results are calculated only for the baseline AM HTO and CM HTO.

This uses a custom function to walk through the supply chain until specified activities are reached, to then report all impacts upstream of those activities. The reporting activities are imported from `final_activities.py`. Ideally this list will be complete, but if not, the algorithm will keep looking for somewhere to stop until a depth or cutoff limit is reached -- which might take longer than expected.

In [1]:
import brightway2 as bw
import numpy as np
import pandas as pd
import bw2calc as bc

In [2]:
from bw_helpers import recursive_calculation
from final_activities import final_activities, component_order

In [3]:
# Load bw databases and activities
bw.projects.set_current("default")
db_AMHTO = bw.Database('AM HTO')
db_CMHTO = bw.Database('CM HTO')
db_UKR = bw.Database('UKR')

AMHTO = db_AMHTO.get("1a637b9baee74199b8027ffeb333279c") 
CMHTO = db_CMHTO.get("2f3acaae6c4e4035b2d24c71725b17d8")
UKR = db_UKR.get("663ac4ef3d314e739f29ec63ea2ca399")
polishing = db_AMHTO.get("8fc93e48b98c4f7487ff73fef8362399_copy1")


print(AMHTO)
print(CMHTO)

my_methods = [
     ('ReCiPe 2016 v1.03, midpoint (H)', 'acidification: terrestrial', 'terrestrial acidification potential (TAP)'),
     ('ReCiPe 2016 v1.03, midpoint (H)', 'climate change', 'global warming potential (GWP1000)'), 
     ('ReCiPe 2016 v1.03, midpoint (H)', 'ecotoxicity: freshwater', 'freshwater ecotoxicity potential (FETP)'),
     ('ReCiPe 2016 v1.03, midpoint (H)', 'ecotoxicity: marine', 'marine ecotoxicity potential (METP)'),
     ('ReCiPe 2016 v1.03, midpoint (H)', 'ecotoxicity: terrestrial', 'terrestrial ecotoxicity potential (TETP)'),
     ('ReCiPe 2016 v1.03, midpoint (H)', 'energy resources: non-renewable, fossil', 'fossil fuel potential (FFP)'),
     ('ReCiPe 2016 v1.03, midpoint (H)', 'eutrophication: freshwater', 'freshwater eutrophication potential (FEP)'),
     ('ReCiPe 2016 v1.03, midpoint (H)', 'eutrophication: marine', 'marine eutrophication potential (MEP)'),
     ('ReCiPe 2016 v1.03, midpoint (H)', 'human toxicity: carcinogenic', 'human toxicity potential (HTPc)'),
     ('ReCiPe 2016 v1.03, midpoint (H)', 'human toxicity: non-carcinogenic', 'human toxicity potential (HTPnc)'),
     ('ReCiPe 2016 v1.03, midpoint (H)', 'ionising radiation', 'ionising radiation potential (IRP)'),
     ('ReCiPe 2016 v1.03, midpoint (H)', 'land use', 'agricultural land occupation (LOP)'),
     ('ReCiPe 2016 v1.03, midpoint (H)', 'material resources: metals/minerals', 'surplus ore potential (SOP)'),
     ('ReCiPe 2016 v1.03, midpoint (H)', 'ozone depletion', 'ozone depletion potential (ODPinfinite)'),
     ('ReCiPe 2016 v1.03, midpoint (H)', 'particulate matter formation', 'particulate matter formation potential (PMFP)'),
     ('ReCiPe 2016 v1.03, midpoint (H)', 'photochemical oxidant formation: human health', 'photochemical oxidant formation potential: humans (HOFP)'),
     ('ReCiPe 2016 v1.03, midpoint (H)', 'photochemical oxidant formation: terrestrial ecosystems', 'photochemical oxidant formation potential: ecosystems (EOFP)'),
     ('ReCiPe 2016 v1.03, midpoint (H)', 'water use', 'water consumption potential (WCP)'),
]

'Implantation' (unit, GB, None)
'Implantation' (unit, GB, None)


In [4]:
lca_obj = bc.LCA({UKR:1}, my_methods[1]) 
lca_obj.lci(factorize=True)
lca_obj.lcia()


In [5]:
from bw_helpers import ScoreGrouper
grouper = ScoreGrouper(lca_obj) 
scores = grouper(final_activities) 
scores

{'Implant (material)': 21.117147377603686,
 'Implant (manufac.)': 4.826972154848034,
 'Argon': 0.0010613074689159456,
 'Instruments (material)': 0.6940001533891245,
 'Instruments (manufac.)': 0.023099105513932287,
 'Packaging': 0.3459041509388158,
 'Sterilisation': 0.5901247271662406,
 'Transport': 3.7972760049610144,
 'Anesthesia': 3.6799758330688896}

In [2]:


#AMHTO_contribs = [recursive_calculation(AMHTO, final_activities, m, max_level =8, cutoff=1e-8) for m in my_methods]
#CMHTO_contribs = [recursive_calculation(CMHTO, final_activities, m,  max_level =8, cutoff=1e-8) for m in my_methods]
#UKR_contribs = [recursive_calculation(UKR, final_activities, m, max_level= 8,cutoff=1e-8) for m in my_methods]
#polishing_contribs = [recursive_calculation(polishing, final_activities, m, max_level =8, cutoff=1e-4) for m in my_methods]


In [7]:
def melt_contribs(contribs, label):
    df = (
        pd.DataFrame(contribs, index=[label for _, label, _ in my_methods])
            [component_order]
            .reset_index(names="method")
            .melt(id_vars=["method"], var_name="component", value_name="score")
    )
    df["device"] = label
    return df.fillna(0)

In [8]:
contribs = pd.concat([
    melt_contribs(CMHTO_contribs, "CM HTO"),
    melt_contribs(AMHTO_contribs, "AM HTO"),
    melt_contribs(UKR_contribs, "UKR"),
], ignore_index=True)

method_ref_scores = contribs.query("device == 'CM HTO'").groupby("method")["score"].sum()

# Normalise
contribs["normalised_score"] = contribs.apply(lambda row: row["score"] / method_ref_scores.loc[row["method"]], axis=1)

contribs

,method,component,score,device,normalised_score
0,acidification: terrestrial,Implant (material),1.566226e-02,CM HTO,0.665716
1,climate change,Implant (material),4.008664e+00,CM HTO,0.392109
2,ecotoxicity: freshwater,Implant (material),3.246248e-01,CM HTO,0.733401
3,ecotoxicity: marine,Implant (material),4.535347e-01,CM HTO,0.735515
4,ecotoxicity: terrestrial,Implant (material),7.366651e+00,CM HTO,0.229259
...,...,...,...,...,...
481,ozone depletion,Anesthesia,3.336679e-07,UKR,0.119345
482,particulate matter formation,Anesthesia,4.134676e-04,UKR,0.036075
483,photochemical oxidant formation: human health,Anesthesia,1.025534e-03,UKR,0.059342
484,photochemical oxidant formation: terrestrial e...,Anesthesia,1.035459e-03,UKR,0.059117


In [10]:
!mkdir -p results
contribs.to_csv("results/all_impact_category_contributions.csv", index=False)

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file results already exists.
Error occurred while processing: results.
